# Selenium

``Selenium`` is a set of widely popular tools used to automate tasks in browsers. 

It is largely used to test applications, but its usages are not limited to testing. It can also be used to perform
screen scraping and automate repetitive tasks in a browser window.

Selenium 3.0 offers three important tools, `Selenium WebDriver`, `Selenium Server`, and `Selenium IDE`

<div align = "center">
    <img src = "https://3fxtqy18kygf3on3bu39kh93-wpengine.netdna-ssl.com/wp-content/uploads/2020/03/Remote-Webdriver-Implementation.png" height=300>
</div>

### Selenium WebDriver
A `WebDriver` drives a browsers using selenium server.

How it works?

* Selenium set up a server (local)
* A WebDriver convert our requests to format that a browser can understand
* The browser sents our requests and the responses come back throught the chain

### Install 

with *conda*
 * `conda install -c conda-forge selenium`

with *pip*
* `pip install selenium`

Download the `webdriver` for the browser Firefox

* https://github.com/mozilla/geckodriver/releases/download/v0.31.0/geckodriver-v0.31.0-win64.zip
* https://github.com/mozilla/geckodriver/releases/download/v0.31.0/geckodriver-v0.31.0-win32.zip

In `Local Dick C` make a folder called `dmozilla` and paste the file ``exe`` extracted from the zip.

In [2]:
# Selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException, WebDriverException

In [3]:
# Others
import time
from pathlib import Path
from urllib.parse import urljoin
import requests
from lxml.html import fromstring, tostring

In [4]:
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options

In [7]:
from dotenv import dotenv_values

In [10]:
config = dotenv_values('./env/.env')

In [12]:
# Set the path of the webdriver
driver_path = config['DRIVER_PATH']
ser = Service(driver_path)
opt = Options()

# selenium without opening browser 
#  opt.headless = True

# Create a new instance of the Firefox driver
driver = webdriver.Firefox(service=ser, options=opt)

# go to the google home page
driver.get(config['GOOGLE_PATH'])

# find the element that’s name attribute is q (the google search box)
inputElement = driver.find_element(By.NAME, value = "q")

# type in the search
inputElement.send_keys("Cheese!")

# submit the form (although google automatically searches now without submitting)
inputElement.submit()

# the page is ajaxy so the title is originally this:
print(driver.title)

# we have to wait for the page to refresh, the last thing that seems to be updated i
try:
    WebDriverWait(driver, 10).until(lambda driver : driver.title.lower().startswith('ch'))
    # You should see "cheese! - Google Search"
    print(driver.title)
    time.sleep(5)
    driver.quit()

except:
    time.sleep(5)
    driver.quit()

Google
Cheese! - Buscar con Google


## Selenium-WebDriver API Commands and Operations

### Fetching a Page

The first thing you’re likely to want to do with WebDriver is navigate to a page. The normal way to do this is by calling `get`

`driver.get("http://www.google.com")`

To ensure robustness, you need to wait for the element(s) to exist in the page using Explicit and Implicit Waits.

### Locating UI Elements (WebElements)

Locating elements in WebDriver can be done on the WebDriver instance itself or on a WebElement.

Each of the language bindings expose a `Find Element` and `Find Elements` method. 

The first returns a WebElement object otherwise it throws an exception. The latter returns a list of WebElements, it can
return an empty list if no DOM elements match the query.

The `Find` methods take a locator or query object called `By`

*By ID* - This is the most efficient and preferred way to locate an element, but not all have an ID.

*By Name* - Find the input element with matching name attribute.

*By Link Text* - Find the link element with matching visible text.

*By Partial Link Text*

*By Class*- Name 'Class' in this case refers to the attribute on the DOM element (A DOM element is something like a DIV, HTML, BODY element on a page). 

  Often in practical use there are many DOM elements with the same class name, thus finding multiple elements becomes the more practical option over finding the first element

*By Tag Name* 

* `<a>`:  defines a hyperlink, which is used to link from one page to another. The most important attribute of the `<a>` element is the `href` attribute.

* `<div>` : defines a division or a section in an HTML document.

* `<img>`: is used to embed an image in an HTML page. Images are not technically inserted into a web page; images are linked to web pages

* `<ul>`: defines an unordered (bulleted) list. Use the `<ul>` tag together with the `<li>` tag to create unordered lists.

By XPATH


Scraping public data.

INEI, a Peruvian institution that provide statistics data about how Peruvian economy behaviour.    

In [5]:
# Set the path of the webdriver
driver_path = config['DRIVER_PATH']
ser = Service(driver_path)
opt = Options()

# Create a new instance of the Firefox driver
driver = webdriver.Firefox(service=ser, options=opt)

# Go to the google home page
driver.get(config['INEI_PATH'])

time.sleep(3)

# Find the element that’s href attribute contains `ConsultaPorEncuesta`
try:
    inputElement = driver.find_element(
        By.XPATH,
        value="//ul[@id='jsmenu']//a[contains(@href,'ConsultaPorEncuesta')]")

# If the element is not founded, raise an exeception
except NoSuchElementException as e:
    print(e.msg)

# click in the element
inputElement.click()

time.sleep(2)

# close
driver.quit()

Here, we use tag `ul` and their attribute `id` in order to localice a point of reference. Then we pass their child by `li` tag and then, localice `a` tag where the text contains Encuestas.

### User Input - Filling In Forms

The best way to learn to fill forms is creating the forms and see what happen when your fill your owns

In [66]:
from selenium.common import exceptions

In [67]:
# Set the path of the webdriver
driver_path = config['DRIVER_PATH']
ser = Service(driver_path)
opt = Options()

# Create a new instance of the firefox driver
driver = webdriver.Firefox(service=ser, options=opt)

# go to the local host
driver.get(config['LOCAL_HOST'])

# Wait until the webpage loading
time.sleep(5)

# find the name of tag input

first_name = driver.find_element(By.NAME, value='email')

first_name.clear()

first_name.send_keys(config['EMAIL'])


last_name = driver.find_element(By.NAME, value='password')

last_name.clear()

last_name.send_keys(config['PASSWORD'])

try:
    male = driver.find_element(By.XPATH, value='.//input[@value="Male"]')
    male.click()
except exceptions.ElementClickInterceptedException as e:
    print(e)

year_1 = driver.find_element(By.XPATH, value='.//input[@value="1"]')
year_1.click()

select_continents = Select(driver.find_element(by=By.XPATH, value='//select[@name="continents"]'))
# select.deselect_all() # If we need

select_continents.select_by_visible_text("Australia")

time.sleep(10)

driver.quit()

If we complete the form, we can **submit** our answer. (I didn't complete so I didn't submit it)

In [ ]:
element_submit = driver.find_element(by=By.ID, value= "submit")
element_submit.click()

### Downloading the source pages


In [ ]:
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium import webdriver
from urllib.parse import urljoin, urlparse
from lxml.html import fromstring, tostring

def DownloadHtml(url:str):

    driver_path = config['DRIVER_PATH']
    ser = Service(driver_path)
    opt = Options()

    # selenium without opening browser 
    opt.headless = True

    # Create a new instance of the Firefox driver
    driver = webdriver.Firefox(service=ser, options=opt)

    driver.get(url)
    
    return driver.page_source

url = config['LINIO_PATH']

page = DownloadHtml(url=url)

tree = fromstring(page)

for a in tree.xpath('//a[@itemprop="url"]'):
    url_category = urljoin(url, a.get('href'))
    page_category = DownloadHtml(url_category)

    print(urljoin(url, a.get('href')))
    

## Moving Between Windows and Frames

Some web applications have many frames or multiple windows. 

WebDriver supports moving between named windows using the `switch_to` method

`driver.switch_to.frame()`, `driver.switch_to.window()`, or `driver.switch_to.alert`

In [1]:
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
from urllib.parse import urljoin, urlparse
import time

In [6]:
driver_path = config['DRIVER_PATH']
ser = Service(driver_path)
opt = Options()

# selenium without opening browser 
 
# opt.headless = True

# Create a new instance of the Chrome driver
driver = webdriver.Firefox(service=ser, options=opt)

# Go to the Inkafarma home page
driver.get(config['INKA_PATH'])

# Wait until the webpage loading
time.sleep(5)

# To check how many iframes have a we we pass in the console:  $x("//iframe")
# Find the element that's name attribute
inputElement = driver.find_element(By.XPATH, value = '//iframe[@id="Braindw2"]')
time.sleep(3)

# Switch to other frame
driver.switch_to.frame(inputElement)
time.sleep(3)

# Find the img
elements = driver.find_elements(by = By.XPATH, value='//a//img')
time.sleep(3)

# dowload links of img
links = set()
for element in elements:
    links.add(element.get_attribute('data-src'))
    #print(element.parent)
    #print(element.text)

time.sleep(5)

# exit
driver.quit()

In [7]:
links

{None,
 'https://dcuk1cxrnzjkh.cloudfront.net/imagesproducto/004080X.jpg',
 'https://dcuk1cxrnzjkh.cloudfront.net/imagesproducto/010197X.jpg',
 'https://dcuk1cxrnzjkh.cloudfront.net/imagesproducto/010754X.jpg',
 'https://dcuk1cxrnzjkh.cloudfront.net/imagesproducto/010767X.jpg',
 'https://dcuk1cxrnzjkh.cloudfront.net/imagesproducto/011592X.jpg',
 'https://dcuk1cxrnzjkh.cloudfront.net/imagesproducto/023735X.jpg',
 'https://dcuk1cxrnzjkh.cloudfront.net/imagesproducto/023736X.jpg',
 'https://dcuk1cxrnzjkh.cloudfront.net/imagesproducto/023738X.jpg',
 'https://dcuk1cxrnzjkh.cloudfront.net/imagesproducto/029892X.jpg',
 'https://dcuk1cxrnzjkh.cloudfront.net/imagesproducto/032717X.jpg',
 'https://dcuk1cxrnzjkh.cloudfront.net/imagesproducto/032801X.jpg',
 'https://dcuk1cxrnzjkh.cloudfront.net/imagesproducto/032806X.jpg',
 'https://dcuk1cxrnzjkh.cloudfront.net/imagesproducto/035630X.jpg',
 'https://dcuk1cxrnzjkh.cloudfront.net/imagesproducto/036051X.jpg',
 'https://dcuk1cxrnzjkh.cloudfront.net/im

### window_handles

* This is used for *navigation* or *multiple windows*
* Python Selenium provides option to handle multiple windows using `window_handles`. Python Selenium WebDriver assigns an `id` to each window as soon as the WebDriver object is instantiated or new window is opened using a WebDriver object. This unique id is known as window handles.

Here more information: http://allselenium.info/handling-multiple-windows-python-selenium/

In [16]:
driver_path = config['DRIVER_PATH']
ser = Service(driver_path)
opt = Options()

# selenium without opening browser

# opt.headless = True

# Create a new instance of the Chrome driver
driver = webdriver.Firefox(service=ser, options=opt)

# Go to the Inkafarma home page
driver.get(config['VITE_PATH'])

# Wait until the webpage loading
time.sleep(5)

# The id for the current webpage
before = driver.window_handles[0]
print(before)
time.sleep(3)

got_to_service = driver.find_element(
    by=By.LINK_TEXT,
    value=
    "native ESM"
)
got_to_service.click()
time.sleep(2)


# After click in window_handles have been added a new id. this is the new windows
all_windows = driver.window_handles
print(all_windows)

if len(all_windows) >=2 :
    print(all_windows[1])

time.sleep(4)
# To switch of original page
driver.switch_to.window(before)

time.sleep(5)
driver.quit()

e080c463-acf3-41a9-8145-4540ddca424a
['e080c463-acf3-41a9-8145-4540ddca424a', '2247eb4e-50bb-4435-a277-9b3f68449f64']
2247eb4e-50bb-4435-a277-9b3f68449f64


### Navigation: History and Location

Python doesn’t have `driver.navigate`. To reiterate: `navigate().to()` and `get()` do exactly the same thing.

The `navigate` interface also exposes the ability to move backwards and forwards in your browser’s history:

In [ ]:
driver.get()
driver.forward()
driver.back()

### Changing the User Agent

About to finish 

### Drag And Drop

About to finish 

In [ ]:
from selenium.webdriver.common.action_chains import ActionChains

element = driver.find_element_by_name("source")
target = driver.find_element_by_name("target")
ActionChains(driver).drag_and_drop(element, target).perform()

# WEBDRIVER: ADVANCED USAGE

##  Explicit and Implicit Waits

### Explicit wait

Explicit waits are available to Selenium clients for imperative, procedural languages. They allow your code to halt program execution, or freeze the thread, until the condition you pass it resolves. The condition is called with a certain frequency until the timeout of the wait is elapsed. This means that for as long as the condition returns a falsy value, it will keep trying and waiting.

They make a good fit for synchronising the state between the `browser` and `its DOM`, and your `WebDriver script`.

An explicit waits is code you define to wait for a certain condition to occur before proceeding further in the code.

There are some convenience methods provided that help you write code that will wait only as long as required.

`WebDriverWait` +  `until` method is one way this can be accomplished.

Recall, `until(function)` is method that *calls* the function provided with the driver as an argument until the return value does not evaluate to False.

`wait` utility ignores no such element errors by default, so we can refactor our instructions to be more concise using `assert`

In [22]:
from selenium.webdriver.support.wait import WebDriverWait

driver_path = config['DRIVER_PATH']
ser = Service(driver_path)
opt = Options()

# selenium without opening browser

# opt.headless = True

# Create a new instance of the Chrome driver
driver = webdriver.Firefox(service=ser, options=opt)

driver.get(config['SELENIUM_PATH'])


# the meaning of `d` is driver
header_h2 = WebDriverWait(driver, timeout=20).until(
    lambda d:d.find_element(By.XPATH, '//h2[@id="explicit-wait"]'))

# assert 
assert header_h2.text == "Explicit wait" , "The page don't contain the title"

time.sleep(2)

driver.quit()

Explicit wait



### Expected conditions

Because it is quite a common occurrence to have to synchronise the DOM and your instructions, most clients also come with a set of predefined expected conditions.

* alert is present
* element exists
* element is visible
* title contains
* title is
* element staleness
* visible text

In [19]:
from selenium.webdriver.support.wait import WebDriverWait

driver_path = config['DRIVER_PATH']
ser = Service(driver_path)
opt = Options()

# selenium without opening browser

# opt.headless = True

# Create a new instance of the Chrome driver
driver = webdriver.Firefox(service=ser, options=opt)

driver.get(config['SELENIUM_PATH'])


is_title_contains = WebDriverWait(driver, timeout=10).until(
    EC.title_contains(title="Waits | Selenium"))

assert is_title_contains , "The page don't contain the title"

time.sleep(2)

driver.quit()

In [15]:
driver_path = config['DRIVER_PATH']
ser = Service(driver_path)
opt = Options()

# selenium without opening browser
opt.headless = True

# Create a new instance of the Chrome driver
driver = webdriver.Firefox(service=ser, options=opt)

driver.get(config['INEI_PATH'])

try:
    element = WebDriverWait(driver, 5)\
        .until(EC.element_to_be_clickable((By.XPATH, "//ul[@id='jsmenu']//a[contains(@href,'ConsultaPorEncuesta')]")))
    img_driver = driver.get_screenshot_as_png()
    with open('screen_shot.png', 'wb') as writer:
        writer.write(img_driver)
    element.click()
except (WebDriverException, NoSuchElementException, TimeoutException,
        ElementClickInterceptedException) as e:
    print(e.msg)
    # print('The element is not clickable')

time.sleep(5)
driver.quit()

This waits up to 10 seconds before throwing a TimeoutException or if it finds the element will return it in 0 - 10 seconds. 

`WebDriverWait` by default calls the `ExpectedCondition` every 500 milliseconds until it returns successfully. 

A successful return is for `ExpectedCondition` type is Boolean return true or not null return value for all other `ExpectedCondition` types.

If the element is `clickable` then, it will return `True`

We can tell to webdriver that waiting a certain amount of time when trying to find an element or elements if they are not immediately available.

This set only once.

In [ ]:
from selenium import webdriver

ff = webdriver.Firefox()
ff.implicitly_wait(10) # seconds
ff.get("http://somedomain/url_that_delays_loading")
myDynamicElement = ff.find_element_by_id("myDynamicElement")

### Execute javascript

Executes JavaScript code snippet in the current context of a selected frame or window.

* `driver.execute_script('return something')`
  * Instead something we can pass some *text*, *document.getElementById('id').value*,  last will return the value of the element thta have this id.
  * We can assing values, like `driver.execute_script("document.getElementById('id').value='text'")`
  * We can click some values, `driver.execute_script("arguments[0].click()", element)`

In [16]:
import time
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [18]:
driver_path = config['DRIVER_PATH']
ser = Service(driver_path)
opt = Options()

# selenium without opening browser 
 
opt.headless = True

driver = webdriver.Firefox(service=ser, options=opt)
driver.implicitly_wait(5)

driver.get(config['ESIKA_PATH'])

elt_joy = driver.find_element(By.LINK_TEXT, 'Joyería')
driver.execute_script('arguments[0].click();', elt_joy)

time.sleep(2)

driver.quit()

In [20]:
driver_path = config['DRIVER_PATH']
ser = Service(driver_path)
opt = Options()

# selenium without opening browser 
 
opt.headless = True

driver = webdriver.Firefox(service=ser, options=opt)
driver.implicitly_wait(5)

driver.get(config['ESIKA_PATH'])

title_page = driver.execute_script('return document.title;')
print(title_page)

time.sleep(2)

driver.quit()

ésika Perú - Todo lo que necesitas para crear tu look


In [21]:
driver_path = config['DRIVER_PATH']
ser = Service(driver_path)
opt = Options()

# selenium without opening browser 
 
opt.headless = True

driver = webdriver.Firefox(service=ser, options=opt)
driver.implicitly_wait(5)

driver.get(config['ESIKA_PATH'])

time.sleep(1)
driver.execute_script('history.go(0);')
print('Refreshing the Web Page')

time.sleep(1)
driver.quit()

In [58]:
# ge all text visible 
driver_path = config['DRIVER_PATH']
ser = Service(driver_path)
opt = Options()

# selenium without opening browser 
 
opt.headless = True

driver = webdriver.Firefox(service=ser, options=opt)
driver.implicitly_wait(5)

driver.get(config['ESIKA_PATH'])

time.sleep(1)

text_visible = driver.execute_script('return document.documentElement.innerText;')
text_to_show = text_visible.replace('\n', '|')
print(text_to_show[:int(len(text_to_show)*0.09)])

time.sleep(1)
driver.quit()

Compra a través de una consultora online|Quiero ser consultora||||0|Favoritos| |Regalos| |Maquillaje| |Perfumes| |Cuidado personal| |Cuidado de la piel| |Joyería| |Blog| |Ofertas||25% off en cuidado de la piel y cuidado personal. Cupón: COMPRUEBAX25||Los Favoritos||Vibranza


In [41]:
driver_path = config['DRIVER_PATH']
ser = Service(driver_path)
opt = Options()

# selenium without opening browser 
 
opt.headless = True


driver = webdriver.Firefox(service=ser, options=opt)
driver.implicitly_wait(5)

driver.get(config['ESIKA_PATH'])

# Find the element
elt_joy = driver.find_element(By.XPATH, "//div[@class='yCmsComponent js_nav__link']//a[contains(@href,'esika-07')]")

# Scroll until the found element
driver.execute_script("arguments[0].scrollIntoView(true);", elt_joy)
time.sleep(1)

# highlighted the border in red
driver.execute_script('arguments[0].style.border = "3px solid red";', elt_joy)
time.sleep(2)

driver.quit()

In [42]:
driver_path = config['DRIVER_PATH']
ser = Service(driver_path)
opt = Options()

# selenium without opening browser 
 
opt.headless = True


driver = webdriver.Firefox(service=ser, options=opt)
driver.implicitly_wait(5)

driver.get(config['ESIKA_PATH'])

driver.execute_script('document.getElementById("js-site-search-input").value = "ofertas";')
time.sleep(2)

driver.quit()

[1.] https://www.youtube.com/c/NaveenAutomationLabs/videos

[2.] https://www.youtube.com/watch?v=OISEEL5eBqg&list=PLzMcBGfZo4-n40rB1XaJ0ak1bemvlqumQ&index=5

### ActionChains

In [47]:
driver_path = config['DRIVER_PATH']
ser = Service(driver_path)
opt = Options()

# selenium without opening browser 
 
opt.headless = True

from selenium.webdriver.common.action_chains import ActionChains

driver = webdriver.Firefox(service=ser, options=opt)
driver.implicitly_wait(5)
driver.get(config['ESIKA_PATH'])

elements_head = driver.find_elements(By.XPATH, "//div[@class='yCmsComponent js_nav__link']//a")
actions = ActionChains(driver)

for element in elements_head:
    try:
        actions.move_to_element(element)
        actions.click_and_hold() # simulate click, but hold in the orinal page
        actions.perform()
        
    except:
        print(element)

time.sleep(2)

driver.quit()

## Using Selenium to write tests

In [14]:
import unittest
import time
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

class PythonOrgSearch(unittest.TestCase):

    def setUp(self):
        driver_path = config['DRIVER_PATH']
        ser = Service(driver_path)
        opt = webdriver.FirefoxOptions()

        # Create a new instance of the Chrome driver
        self.driver = webdriver.Firefox(service=ser, options=opt)

    def test_search_in_python_org(self):
        driver = self.driver
        driver.get(config['PYTHON_PATH'])
        print(driver.title, 'driver.title')
        
        self.assertIn("Python", driver.title)
        elem = driver.find_element(by=By.NAME, value="q")
        elem.send_keys("pycon")
        
        print(Keys.RETURN, 'Keys.RETURN')
        
        elem.send_keys(Keys.RETURN)
        self.assertNotIn("No results found.", driver.page_source)


    def tearDown(self):
        self.driver.close()

In [15]:
if __name__ == "__main__":
    pol = PythonOrgSearch()
    pol.setUp()
    pol.test_search_in_python_org()
    time.sleep(5)
    pol.tearDown()

Welcome to Python.org driver.title
 Keys.RETURN


# Application

The purpose of this application is extracting all images of products of the this page to clone it.
 
Steps - Pseudo Code
* If not exist the `page_html.html`(the web page that we'll download)
  * Go to `https://www.metro.pe/panaderia-y-pasteleria`
  * Localice the class element `titulo-sessao` and go to it
  * We know when we scroll down the page loads more data, so we need to scroll down, but this must be controlled. Here we scroll down each 500px, but this can be changed and then to wait 5 seconds to load more data.
  * If the page don't load any data, the page will be downloaded and the while loop will be dropped.
* If exist the html file
  * Create a model data for the products with pydantic
  * Read the html file with utf-8 encoding
  * Convert the html file to tree, so we can use xpath to find the `items`
  * Each element has `data-id`, `data-price`, ...
  * These data are extracted and append in a list to be converted in a dataframe.

In [ ]:
# selenium
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
# To extract the data
from pathlib import Path
import time
from lxml.html import fromstring
# To model
from pydantic import BaseModel
import pandas as pd

In [1]:
print(50)

50


In [ ]:
if not Path('page_html.html').exists():
    driver_path = config['DRIVER_PATH']
    ser = Service(driver_path)

    # Create a new instance of the Chrome driver
    driver = webdriver.Firefox(service=ser)
    
    # go to the metro.pe>panaderia-y-pasteleria 
    driver.get(config['METRO_PATH'])
    time.sleep(3)

    try:
        inputElement = driver.find_element(
            By.CLASS_NAME, value='titulo-sessao')
        driver.execute_script("arguments[0].scrollIntoView();", inputElement)
        time.sleep(2)
        constant = 500  # 500px
        while True:
            driver.execute_script(f"window.scrollTo(0, {constant})")
            constant += 500
            time.sleep(5)
            Height = driver.execute_script(
                "return document.body.scrollHeight;")
            if constant >= Height:
                print(Height, constant)
                page_html = driver.page_source

                with open('page_html.html', mode='w', encoding='utf-8') as files:
                    files.write(page_html)
                    # pass
                break
            else:
                pass

    except NoSuchElementException as e:
        print(e.msg)

    time.sleep(2)

    # close
    driver.quit()
else:
    class Products(BaseModel):
        data_id: int = None
        data_price: str = None
        data_brand: str = None
        data_uri: str = None
        data_name: str = None
        data_category: str = None

    total_products = []
    with open('page_html.html', mode='r', encoding='utf-8') as files:
        page_html = files.read()
        tree = fromstring(page_html)
        elements = tree.xpath('//div[contains(@class, "product-item ")]')
        for element in elements:
            products = Products(
                data_id=element.get('data-id'),
                data_price=element.get('data-price'),
                data_brand=element.get('data-brand'),
                data_uri=element.get('data-uri'),
                data_name=element.get('data-name'),
                data_category=element.get('data-category')
            )
            # print(element)
            total_products.append(products.dict())
        try:
            df = pd.DataFrame(total_products)
            price_string = df.data_price.str.extract("(\d.+)", expand=False)
            price_clean = price_string.str.replace(',', '')
            price_numeric = pd.to_numeric(price_clean, errors='coerce')
            df.data_price = price_numeric

            category = df.data_category.apply(lambda x: x.split('/')[-1])
            df.data_category = category
            path_sql = r"C:\Users\LENOVO\Desktop\python_course\Sql_course"
            df.to_csv(path_sql + '\data_metro.csv', index=False)

        except:
            print('There was an error in the covertion to dataframe')